In [1]:
# Run Fine-Tune Llama2-7b on SE paired dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import textwrap

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    "msaad02/llama2_7b_brockport_gpt",
    quantization_config=bnb_config,
    device_map={"": 0},
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained("msaad02/llama2_7b_brockport_gpt")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline(
    task="text-generation", 
    model=model, 
    tokenizer=tokenizer
)

In [7]:
def qa(text: str, full = False):
    # textwrap.dedent gets rid of indenting at the start of each newline
    text = textwrap.dedent(f"""\
        Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

        ### Instruction:
        {text}

        ### Response:
        """)
    
    response = pipe(text, max_length=250, do_sample=True, top_k=50, top_p=0.95, temperature=1.0)
    response = response[0]['generated_text']
    response = response.split("### Response:\n")[1] if not full else response

    return response

In [4]:
print(qa("How do I apply?", full = True))

Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

### Instruction:
How do I apply?

### Response:
You can apply to SUNY Brockport online.


In [5]:
print(qa("What kinds of programs does Brockport offer?", full = True))

Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

### Instruction:
What kinds of programs does Brockport offer?

### Response:
Brockport offers a wide range of programs for students to choose from, including business, education, nursing, art and design, and more.


In [10]:
print(qa("Who can I contact for help with financial aid?", full = True))

Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

### Instruction:
Who can I contact for help with financial aid?

### Response:
You can contact the financial aid office at SUNY Brockport for assistance with financial aid. They are available to answer any questions and help you navigate the financial aid process.


In [12]:
print(qa("Are SAT scores required?", full = True))

Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

### Instruction:
Are SAT scores required?

### Response:
SAT scores are not required for admission, but test scores are used as part of Brockport's holistic review process to make admissions decisions.


In [13]:
print(qa("What is the point of auxiliary courses for Antrhopology?"))

Auxiliary courses for Anthropology are important as they provide additional knowledge and context for understanding the field.


In [14]:
print(qa("What resources are available for faculty and staff on the SUNY Brockport website?"))

The SUNY Brockport website provides resources such as webmail, course schedule, faculty resources, and employee access for faculty and staff.


In [16]:
print(qa("Where can you seek help if you are struggling physically, mentally, or emotionally?", full = True))

Below is an inquiry related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.

### Instruction:
Where can you seek help if you are struggling physically, mentally, or emotionally?

### Response:
Brockport has many resources available for students who are struggling physically, mentally, or emotionally, including Student Counseling Services, the Office of Disability Services, and the Health Center.
